In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from datetime import datetime

In [ ]:
train_dtypes = {'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'is_booking': pd.np.int64,
'cnt': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64,
'hotel_cluster': pd.np.int64}

In [ ]:
%%time
all_train = pd.read_csv('train.csv', dtype=train_dtypes)
#temp_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#all_train = pd.concat(temp_train, ignore_index=True)

In [ ]:
%%time
all_train['orig_destination_distance'] = all_train['orig_destination_distance'].fillna(all_train['orig_destination_distance'].median()).astype(int)
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
all_train = all_train.drop(['date_time','srch_ci','srch_co','is_booking', 'cnt'], 1)

In [ ]:
%%time
#Split into training and test data
#split = int(0.75*len(all_train))
#train = all_train[0:split]
#test  = all_train[split:]

In [ ]:
%%time
features_train = all_train.ix[:,:'hotel_market'] 
labels_train = all_train.ix[:,'hotel_cluster':]
#features_test = test.ix[:,:'hotel_market'] 
#labels_test = test.ix[:,'hotel_cluster':]

In [ ]:
features_train = features_train.values
labels_train = labels_train.values
#features_test = features_test.values
#labels_test = labels_test.values

In [ ]:
%%time
#clf = GaussianNB()
#clf.fit(features_train, labels_train.ravel())
clf = AdaBoostClassifier(n_estimators=20)
clf = clf.fit(features_train, labels_train.ravel())

In [ ]:
%%time
#pred = clf.predict(features_test)

In [ ]:
#print accuracy_score(pred, labels_test.ravel())

In [ ]:
test_dtypes = {'id': pd.np.int64,
'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64}

In [ ]:
%%time
all_test = pd.read_csv('test.csv', dtype=test_dtypes)

In [ ]:
all_test['orig_destination_distance'] = all_test['orig_destination_distance'].fillna(0.0).astype(int)
#all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
#all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
#all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
#Remove dates columns
testing_file = all_test.drop(['id', 'date_time','srch_ci','srch_co'], 1)

In [ ]:
testing_file = testing_file.values

In [ ]:
%%time
probs = pd.DataFrame(clf.predict_proba(testing_file))

In [ ]:
%%time
temp_test = pd.Series([i, r.sort_values(ascending=False)[:5].index.values for i,r in probs.iterrows()])

In [ ]:
type(temp_test)

In [ ]:
temp_test = temp_test.values

In [ ]:
temp_test

In [ ]:
%%time
indices = [a for a,b in temp_test]

In [ ]:
%%time
values = [b for a,b in temp_test]

In [ ]:
%%time
#pred = clf.predict(testing_file)

In [ ]:
submission = pd.DataFrame()

In [ ]:
%%time
submission['id'] = indices
submission['hotel_cluster'] = [' '.join(str(x) for x in y) for y in values]

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)